# Suspend and Resume Conversation Threads

This notebook demonstrates how to **suspend and resume conversation threads** with AI agents using the Microsoft Agent Framework.

## Overview

The ability to suspend and resume conversations is essential for building stateless, scalable applications. This pattern enables:

- **Session persistence** across user logins and devices
- **Stateless application design** without keeping conversations in memory
- **Conversation handoff** between different services or processes
- **Long-running conversations** that span hours, days, or weeks
- **Backup and recovery** of conversation state

## Key Concepts

### Thread Types

The Agent Framework supports two types of conversation threads:

1. **Service-Managed Threads**:
   - Conversation history stored in external services (Azure AI, OpenAI)
   - Serialization contains only thread ID (~50 bytes)
   - Automatically synchronized across instances
   - Recommended for production applications

2. **In-Memory Threads**:
   - Conversation history stored locally in the application
   - Serialization contains full message history
   - Useful for custom storage backends
   - Full control over data storage

### Thread Serialization

The `serialize()` and `deserialize_thread()` methods enable:
- Saving thread state to databases, files, or cache
- Restoring conversations with full context
- Moving conversations between application instances

## 📖 Documentation

For more details, see the official documentation:
- [Multi-Turn Conversations](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python)
- [AgentThread Storage](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python#agentthread-storage)

---

## Prerequisites

Before running this notebook, ensure you have:

1. **Environment variables configured** in `agent-framework/.env`:
   - `OPENAI_API_KEY` (for OpenAI examples)
   - Or `AZURE_AI_PROJECT_ENDPOINT` + credentials (for Azure AI examples)

2. **Required packages installed**:
   ```bash
   pip install agent-framework
   ```

## Setup and Imports

In [ ]:
# Import required libraries
import asyncio
import json
import os
from pathlib import Path
from dotenv import load_dotenv

from agent_framework.openai import OpenAIChatClient
from openai import AsyncAzureOpenAI

# Load environment variables
env_path = Path(__file__).parent.parent / ".env" if "__file__" in globals() else Path("../.env")
load_dotenv(env_path)

# Create Azure OpenAI client to be reused
azure_client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY") or os.getenv("OPENAI_API_KEY"),
    api_version="2024-10-21",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

print("Environment loaded and Azure OpenAI client created")

## Example 1: Suspend-Resume with Service-Managed Threads

Service-managed threads store conversation history in external services like OpenAI or Azure AI. This is the **recommended pattern for production applications**.

### Advantages:
- **Lightweight serialization**: Only thread ID is saved (~50 bytes)
- **Automatic synchronization**: Multiple app instances share the same conversation
- **Scalability**: No need to manage conversation storage
- **Cloud backup**: Conversation history is automatically backed up

### How It Works:
1. Create a thread - service assigns a unique ID
2. Have a conversation - messages stored in the service
3. Serialize - save only the thread ID
4. Deserialize - reconnect to the same thread in the service
5. Resume - continue with full conversation context

In [ ]:
async def suspend_resume_service_managed_thread():
    """Demonstrates how to suspend and resume a service-managed thread."""
    print("=== Suspend-Resume Service-Managed Thread ===")
    print()
    print("📌 Service-managed threads store conversation history in OpenAI/Azure.")
    print("   Serialization only contains the thread ID (lightweight).")
    print()

    # Create an agent using Azure OpenAI
    agent = OpenAIChatClient(
        model_id=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o"),
        async_client=azure_client
    ).create_agent(
        name="Joker",
        instructions="You are good at telling jokes."
    )
    
    print("✓ Agent created")
    print()

    # Start a new thread for the agent conversation.
    # The underlying service (OpenAI) creates and manages the thread
    thread = agent.get_new_thread()

    print("✓ New thread created (service-managed)")
    print()

    # --- Phase 1: Initial Conversation ---
    print("--- Phase 1: Initial Conversation ---")
    print()
    
    query = "Tell me a joke about a pirate."
    print(f"User: {query}")
    response = await agent.run(query, thread=thread)
    print(f"Agent: {response}")
    print()

    # --- Phase 2: Suspend (Serialize) ---
    print("--- Phase 2: Suspending Conversation ---")
    print()
    
    # Serialize the thread state, so it can be stored for later use.
    # For service-managed threads, this is very lightweight (just thread ID)
    serialized_thread = await thread.serialize()

    print("📦 Serialized thread:")
    print(f"   {serialized_thread}")
    print()
    print("💾 Storage options for serialized thread:")
    print("   - Database (SQL, NoSQL)")
    print("   - Session storage (cookies, server sessions)")
    print("   - Cache (Redis, Memcached)")
    print("   - File storage")
    print()
    print(f"💡 Serialized size: ~{len(str(serialized_thread))} bytes")
    print("   (In-memory threads contain full message history)")
    print()

    # --- Phase 3: Resume (Deserialize) ---
    print("--- Phase 3: Resuming Conversation ---")
    print()
    
    # Deserialize the thread state after loading from storage.
    # This reconnects to the same thread in the service
    resumed_thread = await agent.deserialize_thread(serialized_thread)

    print("✓ Thread deserialized and reconnected to service")
    print("✓ Full conversation history available from service")
    print()

    # Continue the conversation - agent has full context from previous messages
    query = "Now tell the same joke in the voice of a pirate, and add some emojis to the joke."
    print(f"User: {query}")
    response = await agent.run(query, thread=resumed_thread)
    print(f"Agent: {response}")
    print()
    
    print("✅ Conversation successfully resumed with full context!")
    print()

In [ ]:
# Run the service-managed thread example
await suspend_resume_service_managed_thread()

## Example 2: Suspend-Resume with In-Memory Threads

In-memory threads store conversation history locally in the application. This pattern provides full control over data storage.

### Characteristics:
- **Full message history**: Serialization includes all messages
- **Larger payload**: Grows with conversation length
- **Custom storage**: You control where and how data is stored
- **Flexibility**: Works with any storage backend

### When to Use:
- Custom message store implementations
- Specific data residency requirements
- Integration with existing data infrastructure
- Offline-first applications

### How It Works:
1. Create a thread - messages stored in memory
2. Have a conversation - messages accumulate locally
3. Serialize - export all messages as JSON
4. Deserialize - restore messages to new thread
5. Resume - continue with restored context

In [ ]:
async def suspend_resume_in_memory_thread():
    """Demonstrates how to suspend and resume an in-memory thread."""
    print("=== Suspend-Resume In-Memory Thread ===")
    print()
    print("📌 In-memory threads store messages locally in the application.")
    print("   Serialization contains the full message history.")
    print()

    # Create an agent with in-memory threading using Azure OpenAI
    agent = OpenAIChatClient(
        model_id=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o"),
        async_client=azure_client
    ).create_agent(
        name="Joker",
        instructions="You are good at telling jokes."
    )

    print("✓ Agent created")
    print()

    # Start a new thread for the agent conversation.
    # Messages will be stored in the application's memory
    thread = agent.get_new_thread()

    print("✓ New thread created (in-memory)")
    print()

    # --- Phase 1: Initial Conversation ---
    print("--- Phase 1: Initial Conversation ---")
    print()
    
    query = "Tell me a joke about a pirate."
    print(f"User: {query}")
    response = await agent.run(query, thread=thread)
    print(f"Agent: {response}")
    print()

    # --- Phase 2: Suspend (Serialize) ---
    print("--- Phase 2: Suspending Conversation ---")
    print()
    
    # Serialize the thread state, so it can be stored for later use.
    # For in-memory threads, this includes the entire message history
    serialized_thread = await thread.serialize()

    print("📦 Serialized thread (first 300 chars):")
    serialized_str = str(serialized_thread)
    print(f"   {serialized_str[:300]}...")
    print()
    print("💾 This serialized data contains:")
    print("   - All user messages")
    print("   - All agent responses")
    print("   - Message metadata (timestamps, roles, etc.)")
    print()
    print(f"💡 Serialized size: ~{len(serialized_str)} bytes")
    print("   (In-memory threads grow with conversation length)")
    print()

    # --- Phase 3: Resume (Deserialize) ---
    print("--- Phase 3: Resuming Conversation ---")
    print()
    
    # Deserialize the thread state after loading from storage.
    # This creates a new in-memory thread with the restored messages
    resumed_thread = await agent.deserialize_thread(serialized_thread)

    print("✓ Thread deserialized with full message history")
    print("✓ All previous messages restored to memory")
    print()

    # Continue the conversation - agent has access to all previous messages
    query = "Now tell the same joke in the voice of a pirate, and add some emojis to the joke."
    print(f"User: {query}")
    response = await agent.run(query, thread=resumed_thread)
    print(f"Agent: {response}")
    print()
    
    print("✅ Conversation successfully resumed with full context!")
    print()

In [ ]:
# Run the in-memory thread example
await suspend_resume_in_memory_thread()

## Running Both Examples

Execute both examples to compare the two threading approaches:

In [ ]:
async def run_all_examples():
    """Run both suspend-resume examples."""
    print("Suspend-Resume Thread Examples")
    print("=" * 60)
    print()
    
    await suspend_resume_service_managed_thread()
    print("=" * 60)
    print()
    
    await suspend_resume_in_memory_thread()
    print("=" * 60)
    print()
    
    print("✅ All examples completed successfully!")

In [ ]:
# Run all examples
await run_all_examples()

## Comparison: Service-Managed vs In-Memory Threads

| Feature | Service-Managed | In-Memory |
|---------|----------------|----------|
| **Serialization Size** | ~50 bytes (thread ID) | Full message history |
| **Storage Location** | External service | Application memory |
| **Scalability** | Excellent (shared across instances) | Limited (per-instance) |
| **Setup Complexity** | Low (service handles it) | Medium (custom storage) |
| **Data Control** | Service-managed | Full control |
| **Cost** | Service API calls | Storage costs |
| **Best For** | Production apps | Custom requirements |

## Key Takeaways

### Service-Managed Threads (Recommended)

✅ **Use when:**
- Building production applications
- Need automatic synchronization across instances
- Want minimal serialization overhead
- Prefer managed infrastructure

⚠️ **Consider:**
- API costs for storage
- Dependency on external service
- Data residency in service cloud

### In-Memory Threads

✅ **Use when:**
- Need custom storage backends
- Have specific data residency requirements
- Want full control over data
- Building offline-first applications

⚠️ **Consider:**
- Larger serialization payloads
- Need to implement storage infrastructure
- Synchronization across instances

## Production Patterns

### 1. Database Storage Pattern
```python
# Serialize and save to database
serialized = await thread.serialize()
await db.save_conversation(
    user_id=user_id,
    conversation_id=conversation_id,
    thread_data=serialized
)

# Load and deserialize from database
thread_data = await db.load_conversation(conversation_id)
thread = await agent.deserialize_thread(thread_data)
```

### 2. Session Storage Pattern
```python
# Save to user session
session['conversation_thread'] = await thread.serialize()

# Restore from session
thread = await agent.deserialize_thread(
    session['conversation_thread']
)
```

### 3. Cache Pattern (Redis)
```python
# Cache serialized thread
serialized = await thread.serialize()
await redis.setex(
    f"thread:{thread_id}",
    3600,  # 1 hour TTL
    json.dumps(serialized)
)

# Retrieve from cache
cached = await redis.get(f"thread:{thread_id}")
thread = await agent.deserialize_thread(json.loads(cached))
```

## Best Practices

1. **Always serialize before closing threads**: Capture state before disposal
2. **Use consistent serialization formats**: JSON is recommended for portability
3. **Implement error handling**: Gracefully handle serialization/deserialization failures
4. **Set expiration policies**: Clean up old conversations automatically
5. **Version your serialization format**: Support format migrations
6. **Compress large payloads**: Reduce storage costs for in-memory threads
7. **Encrypt sensitive data**: Protect conversation content in storage

## Next Steps

- Explore **[Custom Message Stores](2-custom_chat_message_store_thread.ipynb)** for custom backends
- Learn about **[Redis Message Store](3-redis_chat_message_store_thread.ipynb)** for distributed scenarios
- Review **[Azure AI Thread Serialization](1-azure-ai-thread-serialization.ipynb)** for cloud integration
- Read **[Multi-Turn Conversation Documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python)**